In [680]:
n, m, k = 5, 10, 3#500, 200, 15
p = k * m + k * n
l = 10#100

In [681]:
import numpy as np
import random
random.seed(3)
np.random.seed(3)

In [682]:
def L(A, B):
    k1, n = A.shape
    k2, m = B.shape
    assert n == m
    total = 0
    for i in range(k1):
        min_row_diff = np.inf
        for j in range(k2):
            row_diff = np.sum((A[i, :] - B[j, :]) ** 2)
            if row_diff < min_row_diff:
                min_row_diff = row_diff
        total += min_row_diff
    return total

In [683]:
from skscope import layer
import jax.numpy as jnp
import numpy as np

layers = [layer.NonNegative(p)]
for i in range(m):  
    coef = np.zeros(p)
    coef[i*k:i*k+k] = 1.0
    layers.append(layer.LinearConstraint(p, jnp.array(coef)))

preselect = list(range(k*m))

In [684]:
true_H = np.random.uniform(100,size=(k, n))
np.put(true_index:=np.zeros((k, n), dtype=int), np.random.choice(n*k, l, replace=False), 1)
true_H *= true_index
true_W = np.random.uniform(size=(m, k))
true_W /= true_W.sum(axis=1, keepdims=True)

X = true_W @ true_H

In [685]:
true_H

array([[45.47100765,  0.        ,  0.        ,  0.        , 11.59825152],
       [11.2669842 , 87.56705426, 79.48295506, 94.90474687, 56.35982548],
       [ 0.        , 54.77351078,  0.        , 72.42975902, 33.0507647 ]])

In [686]:
def nmf(params):
    W = params[:k*m].reshape(m, k)
    H = params[k*m:].reshape(k, n)
    return jnp.sum((X - W @ H) ** 2)

In [687]:
from skscope import ScopeSolver

solver = ScopeSolver(p, k*m+l, preselect=preselect)
params = solver.solve(nmf, layers=layers, jit=True)
round(params[k*m:].reshape(k, n), 2), nmf(params), nmf(np.concatenate([true_W.flatten(), true_H.flatten()]))

(Array([[  0.       ,   3.73     ,   0.       ,   4.47     ,   2.49     ],
        [  0.       , 197.08     ,   0.       , 245.43999  , 126.619995 ],
        [ 22.06     ,   1.8499999,  36.71     ,   0.       ,   3.59     ]],      dtype=float32),
 Array(469.97394, dtype=float32),
 Array(0., dtype=float32))

In [688]:
len(set(true_H.reshape(-1).nonzero()[0]) & set(np.array(params[k*m:].nonzero()[0]))) / l

0.6

In [689]:
from skscope import BaseSolver
solver = BaseSolver(p, p)
dense_params = solver.solve(nmf, layers=layers, jit=True)
round(dense_params[k*m:].reshape(k, n), 2), nmf(dense_params), nmf(np.concatenate([true_W.flatten(), true_H.flatten()]))

(Array([[ 19.56     ,   3.05     ,  10.98     ,   1.15     ,   6.92     ],
        [  5.2799997,  59.039997 ,   5.73     ,  76.56     ,  37.02     ],
        [ 33.77     , 254.87999  , 158.23999  , 295.44998  , 163.7      ]],      dtype=float32),
 Array(1.1004886e-09, dtype=float32),
 Array(0., dtype=float32))

In [690]:
L(true_H, params[k*m:].reshape(k, n)) / np.sum(true_H ** 2), L(true_H, dense_params[k*m:].reshape(k, n)) / np.sum(true_H ** 2)

(Array(0.8247665, dtype=float32), Array(0.2097329, dtype=float32))